In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
import tensorflow as tf

In [5]:
from solarimage.classifier.nn.dataset import DataSet
from solarimage.classifier.nn import trainer
from solarimage.detector.read_files import get_images_labels

In [8]:
images, labels = get_images_labels("example/cat_dog/train/", gray=True)

In [19]:
images[0].reshape(images[0].shape[0] * images[0].shape[1])

array([167, 167, 168, ...,   1,   1,   1], dtype=uint8)

In [8]:
data_set = DataSet(images, labels)

In [13]:
mnist.train.images

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [14]:
mnist = input_data.read_data_sets("example/MNIST_data/", one_hot=True)

Extracting example/MNIST_data/train-images-idx3-ubyte.gz


Extracting example/MNIST_data/train-labels-idx1-ubyte.gz
Extracting example/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting example/MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
from tensorflow.python.framework import dtypes
mnist_dataset = DataSet(mnist.train.images, mnist.train.labels,reshape=False,dtype=dtypes.uint8)

In [14]:
data_set.images[0].shape

(187000,)

In [9]:
loss_list, weights, bias = trainer.runner(data_set, 100)

IndexError: tuple index out of range

In [10]:
# Test trained model
with tf.Session() as sess:
    # sess.run(tf.global_variables_initializer())
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                        y_: mnist.test.labels}))

NameError: name 'y' is not defined

In [22]:
#define a variable to hold normal random values 
normal_rv = tf.Variable( tf.truncated_normal([2,3],stddev = 0.1))

#initialize the variable
init_op = tf.global_variables_initializer()

#run the graph
with tf.Session() as sess:
    sess.run(init_op) #execute init_op
    #print the random values that we sample
    print (sess.run(normal_rv))

[[ 0.18295154  0.1078015  -0.04624603]
 [ 0.11344106 -0.01048654  0.01129937]]


In [47]:
bias

array([-0.37952802,  0.34663069,  0.09938147, -0.27262148,  0.03827327,
        1.30118704, -0.10074756,  0.61118478, -1.41063714, -0.23312168], dtype=float32)

In [44]:
[v.name for v in tf.trainable_variables() if v.name.split("_")[0] == "weights"][-1]

'weights_3:0'